## Steps for Notebook
1. Getting the datasets (cleveland, census data, acs data) (https://la.arcgis.com/databrowser/index.html), (immigation, emigration) (https://www.census.gov/acs/www/data/data-tables-and-tools/data-profiles/2022/) (**Ethan**)
2. Combine into one dataset via geoenrichment (per block per year) (could be zip_code per year) (**Calvin**)
4. Initial Visualization (hotspots for crime and socioeconomic factors, visualize blocks/zip_codes in Cleveland, show difference in demographics with a hotspot comparing 2010 vs 2020 census) (**Calvin** for hotspot), (**Ethan** for chloropleth maps)
5. Perform correlation analysis with hotspots. How much socioeconomic factors explain the varability in crime rate. (**Ethan**)
6. kNN, Isolation Forest, One-class SVM, Random Forest. Predict the crime rate for a zip-code/block based on socioeconomic factors. Split into a training vs test set (80/20). See which ones have the lowest MSE. (**Ethan** for Random Forest, Isolation Forest) (**Calvin** for kNN and One-class SVM)
8. Comparing the models and detemerining the best one. (**Calvin**)

Finish up to step 4 by Thursday 

## Imports

In [ ]:
# imports
import pandas as pd
import geopandas as gpd
import numpy as np
import os

import censusdata
from census import Census
from us import states

from matplotlib import pyplot as plt
import pygris
import folium

import arcgis
from arcgis.gis import GIS
from arcgis import geometry
from arcgis.geometry import Geometry, SpatialReference
from arcgis.features import FeatureLayer

pd.set_option('display.max_columns', None)

# Part 0: Imports and Initialization

## Functions

In [ ]:

def download_OH_data(var_map, year_start, year_end):
    df_final = None
    
    for yr in range(year_start, year_end + 1):
        c = Census("4977648d549eae5dd6bc0563b7c148db6c44642d", year=yr)
        
        for raw_var, alias in var_map.items():
  
            if raw_var.startswith('DP'):
                data = c.acs5dp.get(
                    ('NAME', raw_var),
                    {'for': 'tract:*', 'in': f'state:{states.OH.fips} county:035'}
                )
            else:
                data = c.acs5.get(
                    ('NAME', raw_var),
                    {'for': 'tract:*', 'in': f'state:{states.OH.fips} county:035'}
                )
            

            df_temp = pd.DataFrame(data)
            df_temp.rename(columns={raw_var: f"{alias}_{yr}"}, inplace=True)

            # Merge it into df_final
            if df_final is None:
                # If this is the first chunk of data, just assign
                df_final = df_temp
            else:
                # Otherwise, merge on the geo-id columns
                df_final = pd.merge(
                    df_final, df_temp,
                    on=['NAME', 'state', 'county', 'tract'],
                    how='outer'  # or 'inner', your choice
                )
                
    return df_final

## Part 1: Getting Data

## Census Data

- Utilizing search functionality to locate variables

In [ ]:
# s = censusdata.search('acs5', 2015, 'label', 'family')
# s

s = censusdata.search('acs5', 2015, 'name', 'B')
s

- Codes of collected variables we will be using

In [ ]:
s = censusdata.search('acs5', 2015, 'label', 'graduate')
s

In [ ]:
'''
Add. codes to find:
 - Immigration/emmigration (how many people are moving to/from the area?)
    - mobility/migration
 - Housing information (how many people own/rent their homes?)
 - Age distributions
 - Familial structures
    - Household size, family size
 - in/out of labor force (different from income?)
 - educational attainment
'''
codes = {
    'B06010_003E': 'income_yes',
    'B06010_004E': 'income_no',
    'B06010_005E': 'income_0_10k',
    'B06010_006E': 'income_10_25k',
    'B06010_007E': 'income_25_35k',
    'B06010_008E': 'income_35_45k',
    'B06010_009E': 'income_45_55k',
    'B06010_010E': 'income_55_65k',
    'B06010_011E': 'income_65_75k',
    'B06010_013E': 'income_over_75k',
    'B06002_001E': 'median_age',
    'B15003_022E': 'bachelors_degree',
    'B19083_001E': 'gini_index' # Measure of index inequality
}

- Running the downloads

In [ ]:
test_code = {'B15003_022E': 'bachelors_degree'}

df = download_OH_data(test_code, 2015, 2017)
df.head()

In [ ]:
url = 'https://services3.arcgis.com/dty2kHktVXHrqO8i/arcgis/rest/services/Crime_Incidents/FeatureServer/0'
layer = FeatureLayer(url)
sdf = layer.query(where="OffenseYear >= 2015 AND OffenseYear <= 2021", out_fields="*", as_df=True)
# udeally we would like to save it to ArcGIS, but we can't :(
sdf.head()

## Part 2: Preprocessing and Combining Data

In [ ]:
df.head()

In [ ]:
year_cols = {}
for col in df.columns:
    parts = col.split('_')
    if len(parts) > 1 and parts[-1].isdigit() and len(parts[-1]) == 4:
        year = parts[-1]
        base_name = '_'.join(parts[:-1])
        if year not in year_cols:
            year_cols[year] = []
        year_cols[year].append((base_name, col))
year_cols # dict of years with lists of sets of column_name without years and column_name with years

In [ ]:
dfs = {}
for year, cols in year_cols.items():
    new_df = df[['NAME', 'state', 'county', 'tract']].copy() 
    new_df['year'] = year
    for base_name, col in cols:
        new_df[base_name] = df[col]
    dfs[year] = new_df

In [ ]:
final_df = pd.concat(dfs.values(), ignore_index=True)

In [ ]:
# Changed data format into keys of name, tract, year instead 
# we need to add "39035" to each census tract
final_df.head()

In [ ]:
sdf.head()

## Part 3: Visualizations

## Part 4: Correlation Analysis